In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import dtt_fiber_processing

In [8]:

data_dir = Path('R:\\3_Histology\Qupath Projects\export measurements Test65')
query_file, data_files = dtt_fiber_processing.parse_datafile_paths(data_dir)
data_file = data_files[0] # We're going to work with one file for now


query = pd.read_csv(query_file)
data = pd.read_csv(data_file)

left_data_index = []
right_data_index = []
total_index = []
classification = data['Classification']


for index, value in classification.items():
    if pd.isna(value):  # The total value lists NaN as its classification
        total_index = index
    elif 'Left' in value:  # If the class includes Left, put it in one bin, otherwise, its certainly a right
        left_data_index.append(index)
    else:
        right_data_index.append(index)

left_data = data.iloc[left_data_index]  # Select only left regions
right_data = data.iloc[right_data_index] # Select only right regions

left_data['Classification'].str.strip(to_strip='Left: ')  # Remove Left/Right strings from classification column
right_data['Classification'].str.strip(to_strip='Right: ')


3       POST
4      COApm
5        COA
6       COAp
9       HATA
       ...  
431       or
432        m
433       sp
434      dtd
435     scwm
Name: Classification, Length: 212, dtype: object